In [1]:
import mitreattack.attackToExcel.attackToExcel as attackToExcel
import mitreattack.attackToExcel.stixToDf as stixToDf
import pandas as pd

# download and parse ATT&CK STIX data
attackdata = attackToExcel.get_stix_data("enterprise-attack")
# techniques_data = stixToDf.techniquesToDf(attackdata, "enterprise-attack")
groupsdata = stixToDf.groupsToDf(attackdata)
# softwareToDf = stixToDf.softwareToDf(attackdata)
# relationships = stixToDf.relationshipsToDf(attackdata)
# cs = stixToDf.campaignsToDf(attackdata)

# rdf = relationships['relationships']
# cdf = cs['campaigns']
# rdf = rdf.loc[rdf['source type'] == 'campaign']
# rdf = rdf.loc[rdf['target type'] == 'group']
# display(cdf)
# display(rdf)


2024-10-11 14:00:48.225 | INFO     | mitreattack.attackToExcel.attackToExcel:get_stix_data:69 - Downloading ATT&CK data from github.com/mitre/cti
parsing relationships for type=group: 100%|██████████| 17899/17899 [00:00<00:00, 48672.04it/s]


In [ ]:

df = groupsdata['groups']
df.drop(columns=['contributors','description', 'version', 'domain', 'STIX ID', 'associated groups citations', 'relationship citations'], inplace=True)
df.rename(columns={"ID":'id', "created":"first_seen", "last modified": "last_seen"})
df.rename(columns={"ID":'id', "created":"first_seen", "last modified": "last_seen"}, inplace=True)
df = df[['id', 'name', 'associated groups', 'url', 'first_seen', 'last_seen']]
df = df.reset_index().drop(columns=['index'])
df.sort_values(by='name', inplace=True, key=lambda col: col.str.lower())
df['first_seen'] = pd.to_datetime(df['first_seen'], format='%d %B %Y')
df['last_seen'] = pd.to_datetime(df['last_seen'], format='%d %B %Y')

# Format the dates to 'yyyy_mm_dd'
df['first_seen'] = df['first_seen'].dt.strftime('%Y-%m-%d')
df['last_seen'] = df['last_seen'].dt.strftime('%Y-%m-%d')

df = df.reset_index().drop(columns=['index'])



In [16]:

display(df)
df.to_csv('threat_actor_groups_aliases.csv', index=False) 

,id,name,associated groups,url,first_seen,last_seen
0,G0018,admin@338,,https://attack.mitre.org/groups/G0018,2017-05-31,2020-03-18
1,G0130,Ajax Security Team,"AjaxTM, Flying Kitten, Operation Saffron Rose,...",https://attack.mitre.org/groups/G0130,2021-04-14,2023-10-09
2,G1024,Akira,"GOLD SAHARA, PUNK SPIDER",https://attack.mitre.org/groups/G1024,2024-02-20,2024-04-08
3,G0138,Andariel,"Onyx Sleet, PLUTONIUM, Silent Chollima",https://attack.mitre.org/groups/G0138,2021-09-29,2024-01-08
4,G1007,Aoqin Dragon,,https://attack.mitre.org/groups/G1007,2022-07-14,2022-10-24
...,...,...,...,...,...,...
143,G0112,Windshift,Bahamut,https://attack.mitre.org/groups/G0112,2020-06-25,2021-04-26
144,G0044,Winnti Group,Blackfly,https://attack.mitre.org/groups/G0044,2017-05-31,2023-03-20
145,G0090,WIRTE,,https://attack.mitre.org/groups/G0090,2019-05-24,2022-04-15
146,G0102,Wizard Spider,"DEV-0193, FIN12, GOLD BLACKBURN, Grim Spider, ...",https://attack.mitre.org/groups/G0102,2020-05-12,2024-04-03
